# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [0]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [0]:
# Initialization

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [0]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [0]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [0]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [0]:
get_ticket_price("London")

In [0]:
weather = {"london": "10 degree", "paris": "20 degree", "tokyo": "30 degree", "berlin": "15 degree"}

def get_weather(destination_city):
    print(f"Tool get_weather called for {destination_city}")
    city = destination_city.lower()
    return weather.get(city, "Unknown")

In [0]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

weather_function = {
    "name": "get_weather",
    "description": "Fetches the current weather for a given city. Call this whenever you need to know the weather. for example when a customer asks 'What's the weather like for this city'",
    "parameters": {
      "type": "object",
      "properties": {
        "destination_city": {
          "type": "string",
          "description": "The name of the city to get weather for."
        }
      },
      "required": ["destination_city"],
      "additionalProperties": False
    }
}

In [0]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}, {"type": "function", "function": weather_function}]

In [0]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_responses = []
    for tool_call in message.tool_calls:
        function_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
    
        if function_name == "get_ticket_price":
            result = get_ticket_price(city)
        elif function_name == "get_weather":
            result = get_weather(city)
    
        # Append tool response in OpenAI format
        tool_responses.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": function_name,
            "content": json.dumps(result)  # Convert result to JSON string
        })
    print(json.dumps(tool_responses, indent=2))
    return tool_responses, city

In [0]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        # loop thru response
        for res in response:
            messages.append(res)
        
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

## With this implemenation, you can either ask for ticket price/weather separately or ask for both ticket and weather at the same time. 
    For example: I want to visit London, can you help me find ticket price and its weather


In [0]:
gr.ChatInterface(fn=chat, type="messages").launch()